[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/milioe/Intro-APIs-Python-ML/blob/main/1_Consumir_APIs.ipynb)

# 🚀 Clase 1: Consumir APIs

**APIs de Python: de cero a ML**

---

## 📋 ¿Qué aprenderás hoy?

1. ¿Qué es una API y por qué nos importa?
2. Consumir la API de INEGI (datos económicos de México)
3. Usar SDKs como `yfinance` para datos financieros
4. Conectar con Gemini (IA) para analizar datos

---


## 🛠️ Instalación de dependencias

Ejecuta esta celda para instalar las librerías que usaremos (Tienes que descomentarla)

In [ ]:
# %pip install requests pandas matplotlib yfinance google-generativeai -q

In [2]:
# Importamos las librerías
import requests
import pandas as pd
import matplotlib.pyplot as plt

print("✅ ¡Librerías cargadas correctamente!")


✅ ¡Librerías cargadas correctamente!


---

# 🍽️ ¿Qué es una API?

## La Analogía del Restaurante

Imagina que estás en un restaurante:

| Restaurante | Mundo de las APIs |
|-------------|-------------------|
| Tú (cliente) | Tu programa/código |
| Mesero | **API** |
| Cocina | Servidor/Base de datos |
| Menú | Documentación de la API |
| Tu orden | **Request** (petición) |
| Tu platillo | **Response** (respuesta) |

**API = Application Programming Interface** (Interfaz de Programación de Aplicaciones)

> 💡 Una API es un **intermediario** que permite que dos sistemas se comuniquen sin conocer los detalles internos del otro.

---

## ¿Por qué nos importa como economistas?

- 📊 **Acceso a datos en tiempo real**: INEGI, Banxico, World Bank, entre otras fuentes
- 🤖 **Automatización**: No más descargar CSVs manualmente
- 🔄 **Reproducibilidad**: Tu análisis siempre usa datos actualizados
- 🚀 **Escala**: Procesar miles de consultas automáticamente


Status code: 200
Respuesta: {'count': 25188, 'name': 'Emilio', 'age': 64}


Según la API, alguien llamado TU_NOMBRE tiene aproximadamente None años


---

# 🇲🇽 Tu Primera API: INEGI

Vamos directo con datos económicos reales de México.

**INEGI** (Instituto Nacional de Estadística y Geografía) tiene una API pública para acceder a indicadores económicos.

📚 [Documentación oficial](https://www.inegi.org.mx/servicios/api_indicadores.html)

## ¿Cómo funciona una llamada a API?

```
https://www.inegi.org.mx/app/api/indicadores/.../INDICATOR/{indicador}/.../BIE/2.0/{token}?type=json
```

| Elemento | Qué es | Ejemplo |
|----------|--------|---------|
| **URL** | Dirección de la API | `https://www.inegi.org.mx/app/api/...` |
| **Parámetros** | Datos que enviamos | `indicador`, `token` |
| **Response** | Lo que nos devuelve | JSON con datos |

| Código de respuesta | Significado |
|---------------------|-------------|
| 200 | ✅ Éxito |
| 401 | 🔐 No autorizado (falta API key) |
| 404 | 🔍 No encontrado |
| 500 | 💥 Error del servidor |


In [ ]:
# Token de INEGI (este es público para pruebas)
TOKEN_INEGI = "3dcc1396-5c9a-a8c1-d29d-6b7d9a849634"

# Indicador: Inflación mensual (INPC)
indicador_inflacion = "628194"

# Construimos la URL
url_inegi = f"https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/{indicador_inflacion}/es/0700/false/BIE/2.0/{TOKEN_INEGI}?type=json"

print("URL construida:")
print(url_inegi)


In [ ]:
# Hacemos la petición
response = requests.get(url_inegi)

print(f"Status: {response.status_code}")

# Obtenemos los datos
datos_inegi = response.json()

# Veamos la estructura (primeros niveles)
print("\nLlaves principales:", datos_inegi.keys())


In [ ]:
# Navegamos hasta los datos de la serie
serie = datos_inegi['Series'][0]['OBSERVATIONS']

# Convertimos a DataFrame
df_inflacion = pd.DataFrame(serie)

# Limpiamos los datos
df_inflacion['TIME_PERIOD'] = pd.to_datetime(df_inflacion['TIME_PERIOD'])
df_inflacion['OBS_VALUE'] = pd.to_numeric(df_inflacion['OBS_VALUE'])

# Renombramos columnas
df_inflacion = df_inflacion.rename(columns={
    'TIME_PERIOD': 'Fecha',
    'OBS_VALUE': 'Inflacion'
})

# Vemos los últimos datos
df_inflacion.tail(10)


In [ ]:
# Graficamos la inflación
plt.figure(figsize=(12, 5))
plt.plot(df_inflacion['Fecha'], df_inflacion['Inflacion'], color='#006847', linewidth=2)
plt.title('📈 Inflación en México (INPC)', fontsize=14, fontweight='bold')
plt.xlabel('Fecha')
plt.ylabel('Variación %')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


### 🎯 Tu turno: Obtén otro indicador

Algunos indicadores disponibles:

| Indicador | Código |
|-----------|--------|
| PIB Trimestral | 493911 |
| Desempleo | 444612 |
| Tipo de cambio | 628542 |
| IGAE (Actividad Económica) | 383152 |

**Modifica el código para obtener el PIB:**


In [ ]:
# 🎯 EJERCICIO: Cambia el indicador para obtener el PIB
indicador = "______"  # <- Pon aquí el código del PIB

url = f"https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/{indicador}/es/0700/false/BIE/2.0/{TOKEN_INEGI}?type=json"

response = requests.get(url)
datos = response.json()

# Extrae los datos y crea un DataFrame
serie = datos['Series'][0]['OBSERVATIONS']
df = pd.DataFrame(serie)

print("Nombre del indicador:", datos['Series'][0]['METADATA'][0]['VALUE'])
df.tail()


---

# 📦 SDKs: El Atajo de los Programadores

## ¿Qué es un SDK?

**SDK = Software Development Kit**

Un SDK es una librería que "envuelve" una API para hacerla más fácil de usar.

| Sin SDK (API cruda) | Con SDK |
|---------------------|----------|
| Construir URLs manualmente | Funciones simples |
| Manejar autenticación | Se encarga solo |
| Parsear JSON | Devuelve DataFrames |
| ~15 líneas de código | ~2 líneas de código |

---

## 📈 yfinance: Datos Financieros en 2 Líneas

`yfinance` es un SDK que conecta con Yahoo Finance para obtener datos de:
- Acciones (AAPL, MSFT, AMZN...)
- Índices (^GSPC, ^MXX...)
- Divisas (USDMXN=X, EURUSD=X...)
- Criptomonedas (BTC-USD, ETH-USD...)


In [ ]:
import yfinance as yf

# Tipo de cambio USD/MXN
ticker = yf.Ticker("USDMXN=X")

# Obtenemos datos históricos (últimos 2 años)
df_tipo_cambio = ticker.history(period="2y")

df_tipo_cambio.tail()


In [ ]:
# ¡Graficamos en 2 líneas!
plt.figure(figsize=(12, 5))
plt.plot(df_tipo_cambio.index, df_tipo_cambio['Close'], color='#1f77b4', linewidth=2)
plt.title('💵 Tipo de Cambio USD/MXN', fontsize=14, fontweight='bold')
plt.xlabel('Fecha')
plt.ylabel('Pesos por dólar')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


### 🎯 Tu turno: Compara dos activos

Algunos tickers interesantes:

| Activo | Ticker |
|--------|--------|
| Peso mexicano | USDMXN=X |
| Real brasileño | USDBRL=X |
| Peso colombiano | USDCOP=X |
| Bitcoin | BTC-USD |
| Oro | GC=F |
| Petróleo | CL=F |
| S&P 500 | ^GSPC |
| IPC México | ^MXX |


In [ ]:
# 🎯 EJERCICIO: Compara el peso mexicano vs el real brasileño

# Peso mexicano
mxn = yf.Ticker("USDMXN=X").history(period="1y")['Close']

# Real brasileño - COMPLETA EL TICKER
brl = yf.Ticker("______").history(period="1y")['Close']  # <- Completa aquí

# Normalizamos a base 100 para comparar
mxn_norm = (mxn / mxn.iloc[0]) * 100
brl_norm = (brl / brl.iloc[0]) * 100

# Graficamos
plt.figure(figsize=(12, 5))
plt.plot(mxn_norm.index, mxn_norm, label='Peso Mexicano', linewidth=2)
plt.plot(brl_norm.index, brl_norm, label='Real Brasileño', linewidth=2)
plt.title('📊 Comparación de Monedas vs USD (Base 100)', fontsize=14, fontweight='bold')
plt.xlabel('Fecha')
plt.ylabel('Índice (Base 100)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


---

# 🤖 Gemini: IA para Analizar Datos

**Gemini** es el modelo de IA de Google. Tiene una API gratuita que podemos usar para:
- Interpretar datos
- Generar resúmenes
- Responder preguntas sobre economía

## Obtener tu API Key (gratis)

1. Ve a [Google AI Studio](https://aistudio.google.com/app/apikey)
2. Crea una API Key
3. Cópiala y pégala abajo


In [ ]:
import google.generativeai as genai

# 🔑 Pon tu API key aquí
GEMINI_API_KEY = "TU_API_KEY_AQUI"

# Configuramos Gemini
genai.configure(api_key=GEMINI_API_KEY)

# Creamos el modelo
modelo = genai.GenerativeModel('gemini-1.5-flash')

print("✅ Gemini configurado correctamente")


In [ ]:
# Probamos con una pregunta simple
respuesta = modelo.generate_content("¿Qué es la inflación? Explícalo en 2 oraciones.")

print(respuesta.text)


## 🔥 Combinando APIs: INEGI + Gemini

Ahora viene lo interesante: vamos a pasarle datos de INEGI a Gemini para que los analice.


In [ ]:
# Tomamos los últimos datos de inflación
ultimos_datos = df_inflacion.tail(12).to_string()

# Creamos el prompt
prompt = f"""
Eres un economista experto en México. Analiza los siguientes datos de inflación (INPC) 
y da un breve análisis de la tendencia. Sé conciso (máximo 3 párrafos).

Datos:
{ultimos_datos}
"""

# Enviamos a Gemini
respuesta = modelo.generate_content(prompt)

print("📊 Análisis de Gemini:")
print("-" * 50)
print(respuesta.text)


### 🎯 Tu turno: Modifica el prompt

Experimenta cambiando el prompt para obtener diferentes análisis:


In [ ]:
# 🎯 EJERCICIO: Modifica el prompt para que Gemini te dé el análisis
# en formato de TWEET (máximo 280 caracteres)

prompt_tweet = f"""
______  # <- Escribe tu prompt aquí

Datos:
{ultimos_datos}
"""

respuesta = modelo.generate_content(prompt_tweet)
print(f"🐦 Tweet ({len(respuesta.text)} caracteres):")
print(respuesta.text)


---

# 🏆 Ejercicio Integrador

Ahora te toca a ti. Combina lo que aprendiste para:

1. Obtener datos del **tipo de cambio** con yfinance
2. Pasarle los datos a **Gemini** para que los analice
3. Pedir que el análisis incluya una **recomendación**


In [ ]:
# 🏆 EJERCICIO INTEGRADOR
# Completa el código para analizar el tipo de cambio con IA

# Paso 1: Obtener datos del tipo de cambio (últimos 30 días)
tipo_cambio = yf.Ticker("______").history(period="1mo")  # <- Completa el ticker

# Paso 2: Preparar los datos como texto
datos_texto = tipo_cambio[['Close']].tail(10).to_string()

# Paso 3: Crear un prompt para Gemini
mi_prompt = f"""
______  # <- Escribe tu prompt pidiendo análisis y recomendación

Datos:
{datos_texto}
"""

# Paso 4: Obtener respuesta
respuesta = modelo.generate_content(mi_prompt)
print(respuesta.text)


---

# 📝 Resumen de la Clase

| Concepto | Descripción |
|----------|-------------|
| **API** | Interfaz para comunicar sistemas |
| **Request** | Petición que enviamos |
| **Response** | Respuesta que recibimos |
| **SDK** | Librería que simplifica una API |
| **API Key** | Llave de acceso/autenticación |

## APIs que usamos hoy:

| API/SDK | Uso | Autenticación |
|---------|-----|---------------|
| INEGI | Datos económicos de México | Token público |
| yfinance | Datos financieros | No requiere |
| Gemini | Inteligencia Artificial | API Key gratuita |

---

## 🚀 Próxima clase: ¡Construir tu propia API!

En la siguiente sesión aprenderás a:
- Crear una API con FastAPI
- Exponer un modelo de Machine Learning
- Publicarla en internet con Replit

---

### 📚 Recursos adicionales

- [Documentación INEGI](https://www.inegi.org.mx/servicios/api_indicadores.html)
- [yfinance en PyPI](https://pypi.org/project/yfinance/)
- [Google AI Studio](https://aistudio.google.com/)
- [APIs para economistas](https://github.com/milioe/APIs-for-economists)

---

**¿Dudas?** ✉️ emilio@milioe.com
